In [1]:
import numpy as np
import torch

In [4]:
#Training Data representad as two matrices input and target. 
#1.Input params(temp,rainfall,humidity)
inputs = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]],dtype='float32')

In [20]:
#Target Yield of apples and oranges for each input. (apples,oranges)
targets = np.array([[56,70],
                    [81,101],
                    [119,133],
                    [22,37],
                    [103,119]],dtype='float32')

In [25]:
# Convert input and targets into tensors
inputs = torch.from_numpy(inputs)

In [26]:
targets = torch.from_numpy(targets)

In [27]:
print(inputs)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])


In [29]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [31]:
#Hand code model from Scratch
#Start with Random weights and biases
# Weight matrix : 2x3 because we have 2 targets and 3 input params

w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)


tensor([[-0.2266,  0.3505,  1.2582],
        [ 0.2593,  0.7462, -0.1283]], requires_grad=True)
tensor([ 1.0846, -1.0095], requires_grad=True)


In [33]:
# Model is a simple function that multiplies the inputs matrix with weight matrx (transposed) and adds the biad vector
# x input "@" : matrix mult and w.t(transpose w). Output is a set of predictions

def model(x):
    return x @ w.t() + b

In [48]:
# Generate Predictions
preds = model(inputs)
print (preds)

tensor([[ 62.1246,  62.3989],
        [ 91.8269,  80.0422],
        [101.3052, 114.1015],
        [ 39.5935,  52.7792],
        [107.1645,  79.5374]], grad_fn=<AddBackward0>)


In [49]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [36]:
# got luck the first time itself. the diff is not large


In [50]:
# Loss Function. Essentially the avg of differnces between the predictions and the actuals. The diff is squared to handle any -ves

# MSE Loss

def mse(t1,t2):
    diff = t1 - t2
    return torch.sum(diff*diff) / diff.numel()

In [51]:
loss = mse(preds,targets)
print(loss)

tensor(345.5154, grad_fn=<DivBackward0>)


In [40]:
# Compute Gradient
loss.backward()


/home/soumyo/.local/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [41]:
import cuda

ModuleNotFoundError: No module named 'cuda'

In [52]:
print(w.grad)

tensor([[  394.9568,    29.6660,   174.4915],
        [-1043.9333, -1599.1771,  -988.5647]])


In [58]:
with torch.no_grad():
    w -= w.grad * 1e-20
    b -= b.grad * 1e-20
    

In [59]:
loss = mse(preds,targets)

In [60]:
print(loss)

tensor(345.5154, grad_fn=<DivBackward0>)


In [61]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [62]:
# Train in batches. Epochs (iterations)
for i in range(100):
    preds = model(inputs)
    loss = mse(preds,targets)
    loss.backward()
    with torch.no_grad():
         w-= w.grad * 1e-5
         b-= b.grad * 1e-5
         w.grad.zero_()
         b.grad.zero_()
        

In [63]:
preds = model(inputs)
loss = mse(preds,targets)
print(loss)

tensor(100.2045, grad_fn=<DivBackward0>)


In [64]:
preds


tensor([[ 58.5879,  73.1120],
        [ 87.5516,  95.9349],
        [104.2536, 139.2187],
        [ 28.7398,  53.6200],
        [106.9629, 101.0110]], grad_fn=<AddBackward0>)

In [65]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [66]:
# Linear Regression using PyTorch Built-ins

In [67]:
import torch.nn as nn


In [68]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [70]:
# Tensordata set allows access to data from inputs and targets as tuples and standad APIs for different types of datasets
from torch.utils.data import TensorDataset


In [71]:
#Define Dataset
train_ds = TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [73]:
#Dataloader helps split training data into batches. Also utlities like shuffling and random sampling

from torch.utils.data import DataLoader

In [76]:
# Define Dataloader Spec

batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle=True)

In [77]:
# Use nn.Linear to automatically initialize weights and biases


In [79]:
#Define Model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[0.5556, 0.3856, 0.5280],
        [0.0557, 0.1256, 0.4407]], requires_grad=True)
Parameter containing:
tensor([-0.0405, -0.0770], requires_grad=True)


In [80]:
list(model.parameters())

[Parameter containing:
 tensor([[0.5556, 0.3856, 0.5280],
         [0.0557, 0.1256, 0.4407]], requires_grad=True),
 Parameter containing:
 tensor([-0.0405, -0.0770], requires_grad=True)]

In [81]:
preds = model(inputs)
preds

tensor([[ 89.0513,  31.3555],
        [118.2358,  44.2507],
        [130.5812,  47.1591],
        [ 92.7414,  27.3145],
        [112.2658,  46.6730],
        [ 89.2213,  31.2857],
        [118.3783,  44.5658],
        [131.6647,  47.6556],
        [ 92.5714,  27.3843],
        [112.2382,  47.0580],
        [ 89.1937,  31.6707],
        [118.4059,  44.1809],
        [130.4388,  46.8439],
        [ 92.7690,  26.9295],
        [112.0958,  46.7428]], grad_fn=<AddmmBackward0>)

In [83]:
# PyTorch Loss Function
import torch.nn.functional as F

In [84]:
# Other loss functions areavaialble in nn.funnctional
loss_fn = F.mse_loss


In [85]:
loss = loss_fn(preds,targets)
print(loss)

tensor(2516.2607, grad_fn=<MseLossBackward0>)


In [86]:
# Instead of manually manipulating weights and biases use PyTorch Optimizer

#Define optimizer. Provide learning rate

opt = torch.optim.SGD(model.parameters(),lr=1e-5)

In [88]:
#Model Training. Utlitity function to train the model in batches for epochs

def fit(num_epochs,model,loss_fn,opt,train_dl):

    #Repeat for given nos of epochs
    for epoch in range(num_epochs):
        #Train with batches of data
        for xb,yb in train_dl:
            # 1. generate predictions
            preds = model(xb)
            # 2.Calculate loss
            loss = loss_fn(preds,yb)
            # 3.Compute gradients
            loss.backward()
            # 4.Adjust parametes using gradients
            opt.step()
            # 5. Reset gradients to zero
            opt.zero_grad()

        # print Progress
        if(epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [89]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100], Loss: 352.8480
Epoch [20/100], Loss: 247.1626
Epoch [30/100], Loss: 96.7314
Epoch [40/100], Loss: 113.6144
Epoch [50/100], Loss: 81.5188
Epoch [60/100], Loss: 50.5130
Epoch [70/100], Loss: 55.2823
Epoch [80/100], Loss: 9.3876
Epoch [90/100], Loss: 14.8025
Epoch [100/100], Loss: 17.3583


In [90]:
preds = model(inputs)

In [91]:
preds


tensor([[ 58.2557,  71.3491],
        [ 81.9862,  99.9846],
        [116.3916, 132.6086],
        [ 28.2027,  43.1833],
        [ 97.4476, 114.3477],
        [ 57.1969,  70.3910],
        [ 81.7445,  99.9167],
        [116.6866, 133.1595],
        [ 29.2615,  44.1414],
        [ 98.2647, 115.2379],
        [ 58.0140,  71.2812],
        [ 80.9274,  99.0266],
        [116.6334, 132.6765],
        [ 27.3856,  42.2932],
        [ 98.5065, 115.3057]], grad_fn=<AddmmBackward0>)

In [92]:
targets


tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [93]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[55.0985, 68.5765]], grad_fn=<AddmmBackward0>)

In [94]:
# The model is trained with minimal loss and can be used to predictons

